In [283]:
from glob import glob
import pandas as pd
import numpy as np
import os
import re

def extract_hall_and_date(filepath):
    # ファイル名だけ取り出す
    filename = os.path.basename(filepath)  # '埼玉県_パラッツォ川越店_2024-07-10.csv'

    # 正規表現で抽出
    match = re.match(r"(.*)_(.+)_(\d{4}-\d{2}-\d{2})\.csv", filename)
    if match:
        prefecture = match.group(1)
        hall_name = match.group(2)
        date = match.group(3)
        return prefecture, hall_name, date
    else:
        raise ValueError("ファイル名の形式が正しくありません")


# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

In [ ]:

file_list = glob(r"C:/python/dataOnline/anaslo_02/csv/埼玉県_パラッツォ川越店_2024*.csv")
print(len(file_list))
df = pd.DataFrame()
for i, file in enumerate(file_list):
    prefecture, hall_name, date = extract_hall_and_date(file)
    df_read = pd.read_csv(file)
    # df_read = df_read[["model_name", "unit_no", "game", "BB", "RB"]]
    df_read["hall_name"] = hall_name
    df_read["date"] = pd.to_datetime(date)
    df_read["medals"] = 0
    df_read["game"] = df_read["game"].replace(",", "", regex=True).astype(int)
    # df_read.to_csv(file)
    # print(df_read.columns)
#     df_read.columns = ["hall_name", "date", 'model_name', 'unit_no', 'game', 'medals', 'BB', 'RB']
    df = pd.concat([df, df_read], axis=0, ignore_index=True)
    
# df = df[["hall_name", "date", 'model_name', 'unit_no', 'game', 'BB', 'RB', 'medals']]
# cols_to_convert = ["medals", "game", "BB", "RB"]
# for col in cols_to_convert:
#     if col in df.columns:
#         df[col] = df[col].replace(",", "", regex=True).astype(int)
df

142


,model_name,unit_no,game,BB,RB,hall_name,date,medals
0,マイジャグラーV,444,4552,19,16,パラッツォ川越店,2024-07-10,0
1,マイジャグラーV,445,681,1,0,パラッツォ川越店,2024-07-10,0
2,マイジャグラーV,446,4213,17,12,パラッツォ川越店,2024-07-10,0
3,マイジャグラーV,447,2319,11,3,パラッツォ川越店,2024-07-10,0
4,マイジャグラーV,448,3967,14,11,パラッツォ川越店,2024-07-10,0
...,...,...,...,...,...,...,...,...
45435,ケロット4,557,1792,6,2,パラッツォ川越店,2024-12-31,0
45436,桃太郎電鉄 ～パチスロも定番!～,675,4314,14,22,パラッツォ川越店,2024-12-31,0
45437,新・必殺仕置人 回胴 CRASH SPEC,682,2677,48,11,パラッツォ川越店,2024-12-31,0
45438,ゾンビランドサガ,793,4249,25,6,パラッツォ川越店,2024-12-31,0


In [295]:
df["BB"] = df["BB"].replace(0, np.nan)
df["RB"] = df["RB"].replace(0, np.nan)
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
print(f'以下の日付のデータが含まれています')
print(df.date.unique()[0])
model_list = df["model_name"].unique()
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}', end=", ")
    
df.head()

データサイズ: 35840 x 14
以下の日付のデータが含まれています
2025-01-01 00:00:00
以下のモデルが含まれています
0: アイムジャグラーEX-TP, 1: マイジャグラーV, 2: ゴーゴージャグラー3, 3: スマスロ北斗の拳, 4: 沖ドキ!GOLD‐30, 5: からくりサーカス, 6: モンキーターンV, 7: Re:ゼロから始める異世界生活 season2, 8: かぐや様は告らせたい, 9: 革命機ヴァルヴレイヴ, 10: ゴッドイーター リザレクション, 11: ファンキージャグラー2, 12: バジリスク～甲賀忍法帖～絆2 天膳 BLACK EDITION, 13: 戦国乙女4 戦乱に閃く炯眼の軍師, 14: 戦姫絶唱シンフォギア 正義の歌, 15: 押忍！番長4, 16: 甲鉄城のカバネリ, 17: ジャグラーガールズ, 18: ダンベル何キロ持てる？, 19: ハッピージャグラーVIII, 20: モンスターハンターライズ, 21: ゴジラ対エヴァンゲリオン, 22: スマスロ炎炎ノ消防隊, 23: スマスロ真・北斗無双, 24: バンドリ！, 25: 痛いのは嫌なので防御力に極振りしたいと思います。, 26: 頭文字D 2nd, 27: 鬼武者3, 28: L南国育ち, 29: ToLOVEるダークネス, 30: ウルトラミラクルジャグラー, 31: キングハナハナ-30, 32: グランベルム, 33: ゲゲゲの鬼太郎 覚醒, 34: スーパービンゴネオ, 35: ハナハナホウオウ～天翔～-30, 36: ルパン三世 大航海者の秘宝, 37: 一方通行 とある魔術の禁書目録, 38: 押忍!番長ZERO, 39: 新鬼武者2, 40: 炎炎ノ消防隊, 41: 聖闘士星矢 海皇覚醒 CUSTOM EDITION, 42: A‐SLOT+ この素晴らしい世界に祝福を！, 43: D4DJ Pachi‐Slot Mix, 44: クランキークレスト, 45: コードギアス反逆のルルーシュ3 C.C.&Kallen ver., 46: ゴールデンカムイ, 47: この素晴らしい世界に祝福を！, 48: チバリヨ2, 49: ニューゲッターマウス, 50: ニューパルサーSP4 with 太鼓の達人, 51: バイオハザー

,hall_name,date,model_name,unit_no,game,BB,RB,BB_rate,RB_rate,Total_rate,month,day,weekday,unit_last
0,パラッツォ川越店,2025-01-01,アイムジャグラーEX-TP,457,623,0.0,2.0,0.0,311.5,0.0,2025-01,1,2,7
1,パラッツォ川越店,2025-01-01,アイムジャグラーEX-TP,458,3031,7.0,11.0,433.0,275.5,168.4,2025-01,1,2,8
2,パラッツォ川越店,2025-01-01,アイムジャグラーEX-TP,459,1568,5.0,4.0,313.6,392.0,174.2,2025-01,1,2,9
3,パラッツォ川越店,2025-01-01,アイムジャグラーEX-TP,460,1515,3.0,0.0,505.0,0.0,0.0,2025-01,1,2,0
4,パラッツォ川越店,2025-01-01,アイムジャグラーEX-TP,461,512,1.0,0.0,512.0,0.0,0.0,2025-01,1,2,1


In [296]:
# 各種ピボットテーブル
def new_func(df, model_name):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    pivot_targets = ["game", "RB_rate", "Total_rate"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values=col,
        aggfunc="sum",
    )
    # 日付列を反転・スライス
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]

    labeled_tables = [("GAME", game), ("RB_RATE", rb_rate), ("TOTAL_RATE", total_rate)]

# ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

# display(game.head(2))
# display(rb_rate.head(2))
# display(total_rate.head(2))

# 列を交互に整列して統合・NaN除去
    interleaved_cols = [col for pair in zip(
        game.columns,
        rb_rate.columns,
        total_rate.columns,
    ) for col in pair]
    merged = pd.concat([game, rb_rate, total_rate], axis=1)[interleaved_cols]
    return merged

model_list = [
    'マイジャグラーV',
    'アイムジャグラーEX-TP',
    'ゴーゴージャグラー3',
    'ファンキージャグラー2', 'ハッピージャグラーVIII', 'ジャグラーガールズ'
    ]
merged_all_model = pd.DataFrame()
for model in model_list:
    merged = new_func(df, model)
    merged_all_model = pd.concat([merged_all_model, merged])

merged_all_model = merged_all_model[~merged_all_model.iloc[:, 2].isna()]
merged_all_model.to_csv("parazzo_data.csv")
merged_all_model.head()


GAME    RB_RATE TOTAL_RATE       GAME    RB_RATE  \
date               2025-04-23 2025-04-23 2025-04-23 2025-04-22 2025-04-22   
model_name unit_no                                                          
マイジャグラーV   444         1836.0      918.0      204.0     7895.0      415.5   
           445         6056.0      403.7      208.8     6216.0      518.0   
           446         2298.0      328.3      176.8     6583.0      411.4   
           447         2857.0      408.1      204.1     2804.0      934.7   
           448         2671.0      667.8      296.8     6587.0      329.4   

                   TOTAL_RATE       GAME    RB_RATE TOTAL_RATE       GAME  \
date               2025-04-22 2025-04-21 2025-04-21 2025-04-21 2025-04-20   
model_name unit_no                                                          
マイジャグラーV   444          141.0     6591.0      313.9      149.8     4590.0   
           445          200.5     4008.0      286.3      133.6     4488.0   
           446          160.6     4836.0      322.4      146.5     2332.0   
           447          233.7     2161.0      270.1      166.2     6033.0   
           448          131.7     5682.0      405.9      153.6     4045.0   

                    ... TOTAL_RATE       GAME    RB_RATE TOTAL_RATE  \
date                ... 2025-01-04 2025-01-03 2025-01-03 2025-01-03   
model_name unit_no  ...                                               
マイジャグラーV   444      ...      177.5     5255.0      350.3      169.5   
           445      ...      135.5     3236.0      462.3      202.2   
           446      ...      130.2     4331.0      541.4      173.2   
           447      ...      157.8     1016.0        0.0        0.0   
           448      ...      154.4     6391.0      581.0      159.8   

                         GAME    RB_RATE TOTAL_RATE       GAME    RB_RATE  \
date               2025-01-02 2025-01-02 2025-01-02 2025-01-01 2025-01-01   
model_name unit_no                                                          
マイジャグラーV   444         2567.0      427.8      256.7     5668.0      333.4   
           445         4854.0      606.8      194.2     1132.0      566.0   
           446         4389.0      313.5      162.6     6066.0      433.3   
           447         2225.0      445.0      247.2     3918.0      489.8   
           448         4380.0      312.9      182.5     2785.0      557.0   

                   TOTAL_RATE  
date               2025-01-01  
model_name unit_no             
マイジャグラーV   444          166.7  
           445          188.7  
           446          202.2  
           447          195.9  
           448          174.1  

[5 rows x 336 columns]